<a href="https://colab.research.google.com/github/moosunny/KAKAO-Tech-Bootcamp/blob/main/week1/cli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KBT 1주차 실습 과제 CLI 프로그램 제작
### 미니게임 던전 탈출 게임

In [10]:
import random
import os
import sys

sys.setrecursionlimit(1000)

### 게임 시작 함수

1. 주요 기능:
- 사용자 이름 입력
- 던전 내 이동 방향기 설명
- 도움말 필요시 제공

In [11]:
def game_start():
    print("Player name 을 입력해주세요: ")
    name = input()
    print()
    print("="*65)
    print()
    print()
    print(" "*7, f"안녕하세요 {name}님!")
    print(" "*7, "미니 던전 탈출 게임에 오신 것을 환영합니다!")
    print()
    print()
    print(" "*30, "Made by kai.choi(최무선)/인공지능")
    print("="*65)
    print()
    print("방향키 조작 방법: ")
    print("- 상: W")
    print("- 하: S")
    print("- 좌: A")
    print("- 우: D")
    print()
    print("- 게임 종료 버튼: Q")
    print("="*65)
    print()
    print("게임이 처음이신 분은 help를 입력해주세요!")
    print("도움말이 필요 없으신 분은 아무 키나 입력하신 후 Enter를 쳐주세요.")
    help = input()
    if help == "help":
        help_player()
    else:
        return

### 도움말

1. 주요 기능:
- game_start 함수에서 'help'를 입력하면 게임 상세 도움말 제공
- 제공 정보: 난이도별 몬스터 공격력, 게임 시작 위치, 던전 내부 아이콘 별 의미

In [12]:
def help_player():
    print()
    print("="*65)
    print("< 게임 도움말 >")
    print()
    print("본 게임은 던전 탈출을 목표로 진행하는 게임입니다.")
    print("난이도는 상,중,하 순서로 본인 역량에 맞게 정해주세요.")
    print("모든 player는 던전의 입구(가장 위 맨왼쪽)부터 시작합니다.")
    print("출구(던전 가장 아래 맨오른쪽)에 도착하면 게임은 성공으로 마무리됩니다(게임 도중 Q를 누르시면 중도포기하실 수 있습니다).")
    print("던전 내부에는 아이템 상자(체력회복 포션 물약), 몬스터가 출몰합니다. 난이도에 따라 몬스터의 공격력이 다르기 때문에 참고 부탁드립니다.")
    print("몬스터의 공격력만큼 체력이 감소됩니다.")
    print("몬스터 공격력: 상(3), 중(2), 하(1)")
    print("체력 회복 포션: 체력 회복 2")
    print()
    print("던전 지도에 player 위치는 'P', 목적지는 'E'로 나타납니다.")
    print("던전에서 이동 가능한 칸은(0), 벽(#), 아이템(?), 몬스터(@)로 확인 가능합니다!")
    print("="*65)
    print()

### 던전 생성 함수
- 각 아이콘이 무작위로 배치되어 생성되기 때문에 탈출 가능여부는 생성 후 확인 필요

In [13]:
def make_dungeon(map_size):
    dungeon = [[ random.choice(['0','#','?','@']) for _ in range(map_size)] for _ in range(map_size)]
    dungeon[0][0] = 'P'
    dungeon[-1][-1] = 'E'
    return dungeon

### 사용자의 편의를 위해 매 이동 시 현재 플레이어의 위치를 확인

In [14]:
def print_dungeon(dungeon):
    for i in range(len(dungeon)):
        for j in range(len(dungeon)):
            if dungeon[i][j] == 'P':
                print("P", end = " ")
            else:
                print(dungeon[i][j], end = " ")
        print()
    print()
    # print(player_hp)

### 던전 생성 후 탈출 가능 맵인지 확인하기 위한 함수 -> DFS 알고리즘 기반의 탐색 알고리즘 구현

In [15]:
def check_dungeon(dungeon, player_hp, monster_attack, potion, n):



    visited = [[False] * (n) for _ in range(n)]

    def dfs(x , y, dungeon, visited, player_hp, map_size):
        if dungeon[y][x] == 'E':
            return player_hp

        visited[y][x] = True

        dy = [0,0,-1,1]
        dx = [-1,1,0,0]

        for k in range(4):
            nx = x + dx[k]
            ny = y + dy[k]

            # 방문 가능 범위 조건 설정
            if not (0 <= nx < map_size) or not (0 <= ny < map_size):
                continue
            if visited[ny][nx] == True:
                continue
            if dungeon[ny][nx] == '#':
                continue

            # player hp 가 다음 loop로 넘어갈 경우 영향을 받지 않기 위함
            new_hp = player_hp

            if dungeon[ny][nx] == '?':
                new_hp += potion
            elif dungeon[ny][nx] == '@':
                new_hp -= monster_attack

            result = dfs(nx,ny, dungeon, visited, new_hp, map_size)

            if result is not None:
                return result

        return None


    final_hp = dfs(0,0, dungeon,visited,player_hp,n)

    # 결과 출력
    if final_hp is not None:
        return 'success'
    else:
        return 'fail'

### 던전 생성 시 사용자가 입력한 난이도에 따른 던전 생성 함수

In [16]:
def generate():
    global player_hp, map_size, dungeon, monster_attack
    print("난이도를 선택해주세요!")
    print("상,중,하 중 하나를 선택하신 후 입력해주세요.")
    level = input()
    print()
    print("< Welcom to dungeon >")
    while True:
      if level == "상":
          map_size = 20

          player_hp = 10
          monster_attack = 3

      elif level == "중":
          map_size = 15

          player_hp = 15
          monster_attack = 2

      elif level == "하":
          map_size = 10

          player_hp = 20
          monster_attack = 1

      else:
          print("잘못된 입력입니다. 다시 입력해주세요.")
          level = input()
          continue

      dungeon = make_dungeon(map_size)

      result = check_dungeon(dungeon, player_hp,monster_attack, 2, map_size)

      if result == 'success':
          print("맵 생성에 성공했습니다!")
          print_dungeon(dungeon)
          return dungeon, player_hp

      elif result == 'fail':
          print("맵 생성에 실패했습니다. 다시 게임 난이도를 선택해주세요.")
          level = input()
          continue

### 게임 중 이동 기능을 담당하는 함수

- 벽('#') 혹은 던전 밖의 범위로 이동할 경우 재입력 요구
- 목적지('E')에 도착할 경우 게임 자동 종료
- 중도 포기를 원할 경우 'q' 입력을 받아 게임을 종료시킬 수 있음

In [17]:
def move(x,y, map_size, dungeon, player_hp, monster_attack):

  nx,ny = 0 , 0

  print("이동할 방향을 입력해주세요: ")
  move = input().lower()
  if move == "w":
    nx, ny = x, y-1
    print("위로 이동합니다.")
  elif move == "s":
    nx,ny = x, y+1
    print("아래로 이동합니다.")
  elif move == "a":
    nx,ny = x-1,y
    print("왼쪽으로 이동합니다.")
  elif move == "d":
    nx,ny = x+1,y
    print("오른쪽으로 이동합니다.")
  elif move == "q":
    print("여기서 포기하실 건가요?? ㅠㅠ")
    print("정말 포기하시길 원하시면 다시 q를 입력해주세요.:")
    give_up = input()
    if give_up == "q":
      print("게임을 종료합니다.")
      # get_ipython().magic("stop")
      sys.exit()  # 셀 실행 중단
    print("잘하셨습니다! 조금만 더 노력해바요.")
    # return x, y, player_hp
  else:
    print("잘못된 입력입니다. 다시 입력해주세요.")
    return x, y, player_hp, dungeon

  if not 0 <= nx < map_size or not 0 <= ny < map_size:
    print("범위를 벗어났습니다.")
    return x, y, player_hp, dungeon
  elif dungeon[ny][nx] == '#':
    print("벽에 부딪혔습니다. ㅠㅠ")
    return x, y, player_hp, dungeon

  if dungeon[ny][nx] == '?':
    player_hp += 2
    print("포션을 획득했습니다!: 현재체력(",player_hp,")")

  elif dungeon[ny][nx] == '@':
    player_hp -= monster_attack
    print("몬스터를 만났습니다!: 현재체력(",player_hp,")")

  if dungeon[ny][nx] == 'E':
    print("목적지에 도착했습니다! 게임을 종료합니다.")
    # get_ipython().magic("stop")  # 셀 실행 중단
    sys.exit()


  dungeon[ny][nx] = 'P'
  dungeon[y][x] = 'X'

  return nx, ny, player_hp, dungeon


In [18]:
game_start()
dungeon, player_hp = generate()

x,y = 0,0
while True:
  x, y, player_hp , dungeon = move(x,y,map_size,dungeon,player_hp,monster_attack)
  print_dungeon(dungeon)
  print(player_hp)

Player name 을 입력해주세요: 
kai



        안녕하세요 kai님!
        미니 던전 탈출 게임에 오신 것을 환영합니다!


                               Made by kai.choi(최무선)/인공지능

방향키 조작 방법: 
- 상: W
- 하: S
- 좌: A
- 우: D

- 게임 종료 버튼: Q

게임이 처음이신 분은 help를 입력해주세요!
도움말이 필요 없으신 분은 아무 키나 입력하신 후 Enter를 쳐주세요.
help

< 게임 도움말 >

본 게임은 던전 탈출을 목표로 진행하는 게임입니다.
난이도는 상,중,하 순서로 본인 역량에 맞게 정해주세요.
모든 player는 던전의 입구(가장 위 맨왼쪽)부터 시작합니다.
출구(던전 가장 아래 맨오른쪽)에 도착하면 게임은 성공으로 마무리됩니다(게임 도중 Q를 누르시면 중도포기하실 수 있습니다).
던전 내부에는 아이템 상자(체력회복 포션 물약), 몬스터가 출몰합니다. 난이도에 따라 몬스터의 공격력이 다르기 때문에 참고 부탁드립니다.
몬스터의 공격력만큼 체력이 감소됩니다.
몬스터 공격력: 상(3), 중(2), 하(1)
체력 회복 포션: 체력 회복 2

던전 지도에 player 위치는 'P', 목적지는 'E'로 나타납니다.
던전에서 이동 가능한 칸은(0), 벽(#), 아이템(?), 몬스터(@)로 확인 가능합니다!

난이도를 선택해주세요!
상,중,하 중 하나를 선택하신 후 입력해주세요.
하

< Welcom to dungeon >
맵 생성에 성공했습니다!
P # 0 @ ? ? ? # @ ? 
0 ? 0 # @ # @ 0 # # 
# @ 0 # # # ? 0 0 # 
# ? ? 0 0 @ ? @ # @ 
? # @ # @ # # ? 0 ? 
@ @ ? ? @ ? ? 0 0 # 
0 # 0 @ # # @ @ # # 
? # # # @ 0 # # ? @ 
? 0 0 ? 0 0 0 ? 0 @ 
# 0 @ # # 0 # ? # E 

이동할 방향을 입력해주세요: 
s
아

SystemExit: 